In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate

In [2]:
def getDemoPaths(dir_path):
    demoPaths = []
    x = 0
    for filename in os.listdir(dir_path):
        demoPaths.append(os.path.join(dir_path, filename))
    return demoPaths

In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    mapData, killData, nadeData, flashData, roundData, damageData = [], [], [], [], [], []
    for x in parsers:
        x.parse()
        mapData.append(x.parse(return_type="df"))
        killData.append(x._parse_kills(return_type="df"))
        nadeData.append(x._parse_grenades(return_type="df"))
        flashData.append(x._parse_flashes(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (mapData, killData, nadeData, flashData, roundData, damageData)

In [5]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [6]:
def calculateKDByPlace(kills, players, side):
    places = []
    temp = kills.copy()
    for x in temp.AttackerAreaName.unique():
        places.append(x)
    ret = {}
    for place in places:
        kill = len(kills[kills['AttackerName'].isin(players)][kills['AttackerAreaName'] == place][kills['AttackerSide'] == side])
        deaths = len(kills[kills['VictimName'].isin(players)][kills['VictimAreaName'] == place][kills['VictimSide'] == side])
        temp2 = {'kills' : kill, 'deaths' : deaths, '+-' : (kill-deaths)}
        ret[place] = temp2
    return ret

In [7]:
dir_path = "./demos/gambitMirage/gambit"

In [65]:
mirageMappingPath = './mappings/mappings/'
mappingFile = os.path.join(mirageMappingPath, os.listdir(mirageMappingPath)[0])

In [339]:
output_path = './damageDifferentials/mirage_six_months/gambitDamageDifferentials.csv'

In [98]:
file = open(mappingFile, 'r')
lines = file.readlines()
file.close()

In [99]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [100]:
mappedPlaceNames = {}
for x in lines:
    if(x != ''):
            mappedPlaceNames[x.split(':')[0]] = x.split(':')[1]

In [104]:
lines

['AbovePit: Top Balcony\n',
 'Admin: T Palace Stairs Peek\n',
 'PalaceInterior : Deep Palace\n',
 'Airplane : T Spawn\n',
 'Alley: Deep T Spawn T Ramp\n',
 'APlatform : Stairs Smoke Platform\n',
 'Atrium : T Spawn A Ramp\n',
 'Attic : A Ramp\n',
 'Back : CLose A Ramp\n',
 'BackAlley: Close Left CT A Ramp\n',
 'BackAlleys : Tetris\n',
 'BackCourtyard : A Site\n',
 'BackDoor : Jungle Plant\n',
 'BackEntrance: A Site Exposed to Stairs Connector Cat\n',
 'BackHall : Triple Box\n',
 'BackOfA : Ninja\n',
 'BackOfB : Default A\n',
 'BackRoad: Firebox\n',
 'Backroom: Under balcony\n',
 'Backway: Under balcony on top of the little box\n',
 'Backyard: Balcony Boaxes\n',
 'Banana : Sandwhich\n',
 'BankExterior : Ticketbooth \n',
 'BankInterior : CT Left Side\n',
 'Barn : Deep CT\n',
 'BarnRoof : CT Spawn\n',
 'Basement : CT Murder Hole\n',
 'Bathroom : CT Outside Market\n',
 'BDoors:Bench Closer To Jungle\n',
 'Beach:Bench CLoser to Tribox\n',
 'Bedroom:Flowerpot CT\n',
 'BigForest:CT Cheese Spot

In [106]:
mappedPlaceNames

{'AbovePit': 'Top Balcony\n',
 'Admin': 'T Palace Stairs Peek\n',
 'PalaceInterior ': 'Deep Palace\n',
 'Airplane ': 'T Spawn\n',
 'Alley': 'Deep T Spawn T Ramp\n',
 'APlatform ': 'Stairs Smoke Platform\n',
 'Atrium ': 'T Spawn A Ramp\n',
 'Attic ': 'A Ramp\n',
 'Back ': 'CLose A Ramp\n',
 'BackAlley': 'Close Left CT A Ramp\n',
 'BackAlleys ': 'Tetris\n',
 'BackCourtyard ': 'A Site\n',
 'BackDoor ': 'Jungle Plant\n',
 'BackEntrance': 'A Site Exposed to Stairs Connector Cat\n',
 'BackHall ': 'Triple Box\n',
 'BackOfA ': 'Ninja\n',
 'BackOfB ': 'Default A\n',
 'BackRoad': 'Firebox\n',
 'Backroom': 'Under balcony\n',
 'Backway': 'Under balcony on top of the little box\n',
 'Backyard': 'Balcony Boaxes\n',
 'Banana ': 'Sandwhich\n',
 'BankExterior ': 'Ticketbooth \n',
 'BankInterior ': 'CT Left Side\n',
 'Barn ': 'Deep CT\n',
 'BarnRoof ': 'CT Spawn\n',
 'Basement ': 'CT Murder Hole\n',
 'Bathroom ': 'CT Outside Market\n',
 'BDoors': 'Bench Closer To Jungle\n',
 'Beach': 'Bench CLoser to Tr

In [97]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [12]:
def extractDamageDifferentials(mapData, players, side):
    ret = {}
    for x in mapData[mapData['AttackerName'].isin(players)].AttackerAreaName.unique():
        damageDone = mapData[mapData['AttackerName'].isin(players)][mapData['AttackerSide'] == side][mapData['AttackerAreaName'] == x].HpDamageTaken.sum()
        damageTaken = mapData[mapData['VictimName'].isin(players)][mapData['VictimSide'] == side][mapData['VictimAreaName'] == x].HpDamageTaken.sum()
        ret[x] = {"Damage Dealt" : damageDone, "Damage Taken" : damageTaken, "Damage Differential" : (damageDone - damageTaken)}
    return ret

In [13]:
def prettyPrintPlayerDamageTable(playerDamages, playerKills, playerNames, side):
    temp = extractDamageDifferentials(playerDamages, playerNames, side)
    temp1 = calculateKDByPlace(playerKills,playerNames, side)
    
    for key, val in temp1.items():
        if(key in temp.keys()):
            temp1[key]['Damage Dealt'] = temp[key]['Damage Dealt']
            temp1[key]['Damage Taken'] = temp[key]['Damage Taken']
            temp1[key]['Damage Differential'] = temp[key]['Damage Differential']
        else:
            temp1[key]['Damage Dealt'] = "N/A"
            temp1[key]['Damage Taken'] = "N/A"
            temp1[key]['Damage Differential'] = "N/A"
            
    prettyPrint = {"Places" : [], "Kills" : [], "Deaths" : [], "KD Diff" : [], "Damage Dealt" : [], "Damage Taken" : [], "Damage Differential" : []}
    
    for key, val in temp1.items():
        prettyPrint['Places'].append(key)
        prettyPrint['Kills'].append(val['kills'])
        prettyPrint['Deaths'].append(val['deaths'])
        prettyPrint['KD Diff'].append(val['+-'])
        prettyPrint['Damage Dealt'].append(val['Damage Dealt'])
        prettyPrint['Damage Taken'].append(val['Damage Taken'])
        prettyPrint['Damage Differential'].append(val['Damage Differential'])
    
    return(tabulate(prettyPrint, headers=prettyPrint.keys(), tablefmt="github"))

In [330]:
def createDifferentialDF(players, damageData, killData, roundData):
    locationNames = []
    playerNames = []
    totalKills = []
    totalDeaths = []
    totalKdDifferential = []
    totalDamageDealt = []
    totalDamageTaken = []
    totalDamageDifferential = []
    ctKills = []
    ctDeaths = []
    ctKdDifferential = []
    ctDamageDealt = []
    ctDamageTaken = []
    ctDamageDifferential = []
    tKills = []
    tDeaths = []
    tKdDifferential = []
    tDamageDealt = []
    tDamageTaken = []
    tDamageDifferential = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            tempTotalKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y])
            tempTotalDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y])
            tempTotalKdDifferential = tempTotalKills - tempTotalDeaths
            
            tempTotalDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y].HpDamageTaken.sum()
            tempTotalDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y].HpDamageTaken.sum()
            tempTotalDamageDifferential = tempTotalDamageDealt - tempTotalDamageTaken
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            tempTotalRounds = 0
                                                    
            for z in killData[killData.AttackerName.isin(players[x])].MatchId.value_counts().index:                                                                                    
                tempTotalRounds += roundData[roundData['MatchId'] == z].iloc[-1].RoundNum
            
            locationNames.append(y)
            playerNames.append(x)
            totalKills.append(tempTotalKills)
            totalDeaths.append(tempTotalDeaths)
            totalKdDifferential.append(tempTotalKdDifferential)
            
            totalDamageDealt.append(tempTotalDamageDealt)
            totalDamageTaken.append(tempTotalDamageTaken)
            totalDamageDifferential.append(tempTotalDamageDifferential)
            
            ctKills.append(tempCTKills)
            ctDeaths.append(tempCTDeaths)
            ctKdDifferential.append(tempCTKdDifferential)
            
            ctDamageDealt.append(tempCTDamageDealt)
            ctDamageTaken.append(tempCTDamageTaken)
            ctDamageDifferential.append(tempCTDamageDifferential)
            
            tKills.append(tempTKills)
            tDeaths.append(tempTDeaths)
            tKdDifferential.append(tempTKdDifferential)
            
            tDamageDealt.append(tempTDamageDealt)
            tDamageTaken.append(tempTDamageTaken)
            tDamageDifferential.append(tempTDamageDifferential)
            
            totalRounds.append(tempTotalRounds)
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Total Kills'] = totalKills
    dataframe['Total Deaths'] = totalDeaths
    dataframe['K/D'] = totalKdDifferential
    dataframe['Total Damage Dealt'] = totalDamageDealt
    dataframe['Total Damage Taken'] = totalDamageTaken
    dataframe['Total Damage Differential'] = totalDamageDifferential
    dataframe['CT Kills'] = ctKills
    dataframe['CT Deaths'] = ctDeaths
    dataframe['CT K/D'] = ctKdDifferential
    dataframe['CT Damage Dealt'] = ctDamageDealt
    dataframe['CT Damage Taken'] = ctDamageTaken
    dataframe['CT Damage Differential'] = ctDamageDifferential
    dataframe['T Kills'] = tKills
    dataframe['T Deaths'] = tDeaths
    dataframe['T K/D'] = tKdDifferential
    dataframe['T Damage Dealt'] = tDamageDealt
    dataframe['T Damage Taken'] = tDamageTaken
    dataframe['T Damage Differential'] = tDamageDifferential
    dataframe['Total Rounds'] = totalRounds
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [332]:
playerDifferentials = createDifferentialDF(player_names, damageData, killData, roundData)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Serie

In [334]:
player_names = {'Sh1ro' : shiro_names, 'Hobbit' : hobbit_names, 'Interz' : interz_names, 'Nafany' : nafany_names, 'ax1le' : ax1le_names}

In [14]:
demoPaths = getDemoPaths(dir_path)

In [15]:
demoPaths

['./demos/gambitMirage/gambit/furia-vs-gambit-m3-mirage_2346946.dem',
 './demos/gambitMirage/gambit/gambit-vs-complexity-m2-mirage_2345614.dem',
 './demos/gambitMirage/gambit/gambit-vs-forze-m1-mirage_2345856.dem',
 './demos/gambitMirage/gambit/gambit-vs-furia-m1-mirage_2347751.dem',
 './demos/gambitMirage/gambit/gambit-vs-havu-m2-mirage_2346895.dem',
 './demos/gambitMirage/gambit/gambit-vs-heroic-m3-mirage_2346579.dem',
 './demos/gambitMirage/gambit/gambit-vs-heroic-m5-mirage_2347028.dem',
 './demos/gambitMirage/gambit/gambit-vs-spirit-m2-mirage_2346894.dem',
 './demos/gambitMirage/gambit/gambit-vs-virtus-pro-m3-mirage_2345809.dem',
 './demos/gambitMirage/gambit/gambit-vs-virtus-pro-m3-mirage_2346388.dem',
 './demos/gambitMirage/gambit/k23-vs-gambit-m1-mirage_2345855.dem',
 './demos/gambitMirage/gambit/natus-vincere-vs-gambit-m2-mirage_2346940.dem',
 './demos/gambitMirage/gambit/nemiga-vs-gambit-m1-mirage_2345805.dem',
 './demos/gambitMirage/gambit/teamone-vs-gambit-m2-mirage_2346943.

In [16]:
len(demoPaths)

15

In [17]:
demoParsers = generateDemoParsers(demoPaths)

14:12:45 [INFO] Go version>=1.14.0
14:12:45 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/furia-vs-gambit-m3-mirage_2346946.dem
14:12:45 [INFO] Setting demo id to furia-vs-gambit-m3-mirage
14:12:45 [INFO] Setting parse rate to 128
14:12:45 [INFO] Go version>=1.14.0
14:12:45 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/gambit-vs-complexity-m2-mirage_2345614.dem
14:12:45 [INFO] Setting demo id to gambit-vs-complexity-m2-mirage
14:12:45 [INFO] Setting parse rate to 128
14:12:46 [INFO] Go version>=1.14.0
14:12:46 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/gambit-vs-forze-m1-mirage_2345856.dem
14:12:46 [INFO] Setting demo id to gambit-vs-forze-m1-mirage
14:12:46 [INFO] Setting parse rate to 128
14:12:46 [INFO] Go version>=1.14.0
14:12:46 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/gambitM

# Demo Parsers: 15


In [18]:
maps, kills, nades, flashs, rounds, damages = generateData(demoParsers)

14:12:47 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:12:47 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/furia-vs-gambit-m3-mirage_2346946.dem
14:13:09 [INFO] Wrote demo parse output to furia-vs-gambit-m3-mirage.json
14:13:09 [INFO] Reading in JSON from furia-vs-gambit-m3-mirage.json
14:13:09 [INFO] JSON data loaded, available in the `json` attribute to parser
14:13:09 [INFO] Successfully parsed JSON output
14:13:09 [INFO] Successfully returned JSON output
14:13:09 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:13:09 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/furia-vs-gambit-m3-mirage_2346946.dem
14:13:28 [INFO] Wrote demo parse output to furia-vs-gambit-m3-mirage.json
14:13:28 [INFO] Reading in JSON from furia-vs-gambit-m3-mirage.json
14:13:29 [INFO] JSON data loaded, available in the `json` attribute to parser
14:13:29 [INFO] Successfully parsed JSON o

14:16:19 [INFO] Parsed kills to Pandas DataFrame
14:16:19 [INFO] Parsed damages to Pandas DataFrame
14:16:19 [INFO] Parsed grenades to Pandas DataFrame
14:16:19 [INFO] Parsed flashes to Pandas DataFrame
14:16:19 [INFO] Parsed bomb_events to Pandas DataFrame
14:16:19 [INFO] Parsed frames to Pandas DataFrame
14:16:19 [INFO] Parsed player frames to Pandas DataFrame
14:16:19 [INFO] Returned dataframe output
14:16:19 [INFO] Parsed kills to Pandas DataFrame
14:16:19 [INFO] Parsed grenades to Pandas DataFrame
14:16:19 [INFO] Parsed flashes to Pandas DataFrame
14:16:19 [INFO] Parsed rounds to Pandas DataFrame
14:16:19 [INFO] Parsed damages to Pandas DataFrame
14:16:19 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:16:19 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/gambit-vs-heroic-m3-mirage_2346579.dem
14:16:38 [INFO] Wrote demo parse output to gambit-vs-heroic-m3-mirage.json
14:16:38 [INFO] Reading in JSON from gambit-vs-heroic-

14:19:26 [INFO] Successfully parsed JSON output
14:19:26 [INFO] Successfully returned JSON output
14:19:26 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:19:26 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/gambit-vs-virtus-pro-m3-mirage_2346388.dem
14:19:50 [INFO] Wrote demo parse output to gambit-vs-virtus-pro-m3-mirage.json
14:19:50 [INFO] Reading in JSON from gambit-vs-virtus-pro-m3-mirage.json
14:19:51 [INFO] JSON data loaded, available in the `json` attribute to parser
14:19:51 [INFO] Successfully parsed JSON output
14:19:51 [INFO] Successfully returned JSON output
14:19:51 [INFO] Parsed rounds to Pandas DataFrame
14:19:51 [INFO] Parsed kills to Pandas DataFrame
14:19:51 [INFO] Parsed damages to Pandas DataFrame
14:19:51 [INFO] Parsed grenades to Pandas DataFrame
14:19:51 [INFO] Parsed flashes to Pandas DataFrame
14:19:51 [INFO] Parsed bomb_events to Pandas DataFrame
14:19:51 [INFO] Parsed frames to Pandas DataFrame
1

14:22:34 [INFO] Parsed kills to Pandas DataFrame
14:22:34 [INFO] Parsed grenades to Pandas DataFrame
14:22:34 [INFO] Parsed flashes to Pandas DataFrame
14:22:34 [INFO] Parsed rounds to Pandas DataFrame
14:22:34 [INFO] Parsed damages to Pandas DataFrame
14:22:34 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:22:34 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/winstrike-vs-gambit-m2-mirage_2345996.dem
14:22:45 [INFO] Wrote demo parse output to winstrike-vs-gambit-m2-mirage.json
14:22:45 [INFO] Reading in JSON from winstrike-vs-gambit-m2-mirage.json
14:22:45 [INFO] JSON data loaded, available in the `json` attribute to parser
14:22:45 [INFO] Successfully parsed JSON output
14:22:45 [INFO] Successfully returned JSON output
14:22:45 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:22:45 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/gambitMirage/gambit/winstrike-vs-gambit-m2-mirage_23459

In [19]:
failedDemos = []
gambitMaps, gambitKills, gambitNades, gambitFlashs, gambitRounds, gambitDamages = [], [], [], [], [], []
for x in range(len(kills)):
    if(kills[x].shape[0] > 100):
        gambitMaps.append(maps[x])
        gambitKills.append(kills[x])
        gambitNades.append(nades[x])
        gambitFlashs.append(flashs[x])
        gambitRounds.append(rounds[x])
        gambitDamages.append(damages[x])
    else:
        failedDemos.append(x)

In [101]:
killData = concat_data(gambitKills)
nadeData = concat_data(gambitNades)
flashData = concat_data(gambitFlashs)
roundData = concat_data(gambitRounds)
damageData = concat_data(gambitDamages)

In [102]:
killData.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'AssisterSteamId', 'AssisterName', 'AssisterTeam',
       'AssisterSide', 'AssisterX', 'AssisterY', 'AssisterZ', 'AssisterAreaId',
       'AssisterAreaName', 'IsWallbang', 'IsFlashed', 'IsHeadshot', 'IsTrade',
       'PlayerTradedName', 'PlayerTradedTeam', 'PlayerTradedSteamId',
       'IsFirstKill', 'Weapon', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [103]:
mappedPlaceNames

{'AbovePit': 'Top Balcony',
 'Admin': 'T Palace Stairs Peek',
 'PalaceInterior': 'Deep Palace',
 'Airplane': 'T Spawn',
 'Alley': 'Deep T Spawn T Ramp',
 'APlatform': 'Stairs Smoke Platform',
 'Atrium': 'T Spawn A Ramp',
 'Attic': 'A Ramp',
 'Back': 'Close A Ramp',
 'BackAlley': 'Close Left CT A Ramp',
 'BackAlleys': 'Tetris',
 'BackCourtyard': 'A Site',
 'BackDoor': 'Jungle Plant',
 'BackEntrance': 'A Site Exposed to Stairs Connector Cat',
 'BackHall': 'Triple Box',
 'BackofA': 'Ninja',
 'BackofB': 'Default A',
 'BackRoad': 'Firebox',
 'BackRoom': 'Under balcony',
 'BackWay': 'Under balcony on top of the little box',
 'BackYard': 'Balcony Boaxes',
 'Banana': 'Sandwhich',
 'BankExterior': 'Ticketbooth ',
 'BankInterior': 'CT Left Side',
 'Barn': 'Deep CT',
 'BarnRoof': 'CT Spawn',
 'Basement': 'CT Murder Hole',
 'Bathroom': 'CT Outside Market',
 'BDoors': 'Bench Closer To Jungle',
 'Beach': 'Bench Closer to Tribox',
 'Bedroom': 'Flowerpot CT',
 'BigForest': 'CT Cheese Spot Murder Hole 

In [104]:
killData.VictimAreaName.value_counts()

Floor51         102
Attic            94
BankExterior     87
BackofB          77
BigOffice        76
               ... 
Fountain          1
Bedroom           1
APlatform         1
Dumpster          1
Den               1
Name: VictimAreaName, Length: 100, dtype: int64

In [105]:
killData[killData['AttackerAreaName'] == None]

,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName


In [106]:
killData.iloc[310]

Tick                                       30209
Second                                   137.141
AttackerSteamId                              NaN
AttackerName                                None
AttackerTeam                                None
AttackerSide                                None
AttackerX                                    NaN
AttackerY                                    NaN
AttackerZ                                    NaN
AttackerAreaId                               NaN
AttackerAreaName                            None
AttackerViewX                                NaN
AttackerViewY                                NaN
VictimSteamId                  76561198081484775
VictimName                       sh1ro.VulkanBet
VictimTeam                                Gambit
VictimSide                                     T
VictimX                                  -1523.5
VictimY                                 -2141.11
VictimZ                                 -252.515
VictimAreaId        

In [107]:
for x in range(len(killData)):
    if  killData.iloc[x]['AttackerAreaName'] != None:
        killData.at[x, 'VictimAreaName'] = mappedPlaceNames[killData.iloc[x]['VictimAreaName']]
        killData.at[x, 'AttackerAreaName'] = mappedPlaceNames[killData.iloc[x]['AttackerAreaName']]

In [114]:
mappedPlaceNames

{'AbovePit': 'Top Balcony\n',
 'Admin': 'T Palace Stairs Peek\n',
 'PalaceInterior ': 'Deep Palace\n',
 'Airplane ': 'T Spawn\n',
 'Alley': 'Deep T Spawn T Ramp\n',
 'APlatform ': 'Stairs Smoke Platform\n',
 'Atrium ': 'T Spawn A Ramp\n',
 'Attic ': 'A Ramp\n',
 'Back ': 'CLose A Ramp\n',
 'BackAlley': 'Close Left CT A Ramp\n',
 'BackAlleys ': 'Tetris\n',
 'BackCourtyard ': 'A Site\n',
 'BackDoor ': 'Jungle Plant\n',
 'BackEntrance': 'A Site Exposed to Stairs Connector Cat\n',
 'BackHall ': 'Triple Box\n',
 'BackOfA ': 'Ninja\n',
 'BackOfB ': 'Default A\n',
 'BackRoad': 'Firebox\n',
 'Backroom': 'Under balcony\n',
 'Backway': 'Under balcony on top of the little box\n',
 'Backyard': 'Balcony Boaxes\n',
 'Banana ': 'Sandwhich\n',
 'BankExterior ': 'Ticketbooth \n',
 'BankInterior ': 'CT Left Side\n',
 'Barn ': 'Deep CT\n',
 'BarnRoof ': 'CT Spawn\n',
 'Basement ': 'CT Murder Hole\n',
 'Bathroom ': 'CT Outside Market\n',
 'BDoors': 'Bench Closer To Jungle\n',
 'Beach': 'Bench CLoser to Tr

In [137]:
damageData.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'Weapon', 'HpDamage', 'HpDamageTaken', 'ArmorDamage',
       'ArmorDamageTaken', 'HitGroup', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [138]:
for x in range(len(damageData)):
    if  damageData.iloc[x]['AttackerAreaName'] != None:
        damageData.at[x, 'VictimAreaName'] = mappedPlaceNames[damageData.iloc[x]['VictimAreaName']]
        damageData.at[x, 'AttackerAreaName'] = mappedPlaceNames[damageData.iloc[x]['AttackerAreaName']]

In [139]:
killData

,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName
0,7547,30.031250,7.656120e+16,VINI,FURIA,CT,-1491.960449,452.082458,-167.968750,180.0,...,True,False,None,None,NaN,True,USP-S,1,furia-vs-gambit-m3-mirage,de_mirage
1,8150,34.742188,7.656120e+16,interz,Gambit,T,90.629951,-2367.156250,-39.968750,155.0,...,True,False,Ax1Le,Gambit,7.656120e+16,False,p250,1,furia-vs-gambit-m3-mirage,de_mirage
2,8889,40.515625,7.656120e+16,interz,Gambit,T,76.037605,-2083.767334,-35.968750,47.0,...,True,False,KSCERATO,FURIA,7.656120e+16,False,p250,1,furia-vs-gambit-m3-mirage,de_mirage
3,11856,63.695312,7.656120e+16,Hobbit,Gambit,T,278.709717,-1496.237915,-175.968750,60.0,...,True,False,arT,FURIA,7.656120e+16,False,USP-S,1,furia-vs-gambit-m3-mirage,de_mirage
4,12236,66.664062,7.656120e+16,yuurih,FURIA,CT,-504.352875,-400.609680,-168.368683,40.0,...,False,False,VINI,FURIA,7.656120e+16,False,USP-S,1,furia-vs-gambit-m3-mirage,de_mirage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,357571,77.464567,7.656120e+16,-Ax1Le-,Gambit,CT,-684.528137,-312.753571,-159.968750,304.0,...,False,False,sh1ro,Gambit,7.656120e+16,False,AK-47,25,winstrike-vs-gambit-m2-mirage,de_mirage
2367,357639,78.000000,7.656120e+16,Xo66uT,Gambit,CT,-267.162445,-1531.893799,-167.968750,143.0,...,True,False,Lack1,Winstrike,7.656120e+16,False,M4A4,25,winstrike-vs-gambit-m2-mirage,de_mirage
2368,357695,78.440945,7.656120e+16,-Ax1Le-,Gambit,CT,-670.226929,-332.940796,-159.968750,650.0,...,True,False,nealan,Winstrike,7.656120e+16,False,AK-47,25,winstrike-vs-gambit-m2-mirage,de_mirage
2369,357897,80.031496,7.656120e+16,n0rb3r7,Winstrike,T,-47.975136,-1625.312256,-167.978867,145.0,...,False,False,nickelback,Winstrike,7.656120e+16,False,AK-47,25,winstrike-vs-gambit-m2-mirage,de_mirage


In [140]:
killData.VictimAreaName.value_counts()

CT Side Apps                              102
A Ramp                                     93
Ticketbooth                                87
Default A                                  77
Bottom Stairs                              76
                                         ... 
Under balcony on top of the little box      1
BackRoad                                    1
Construction                                1
BackCourtyard                               1
FrontRoom                                   1
Name: VictimAreaName, Length: 108, dtype: int64

In [141]:
killData.head()

,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName
0,7547,30.031250,7.656120e+16,VINI,FURIA,CT,-1491.960449,452.082458,-167.968750,180.0,...,True,False,None,None,NaN,True,USP-S,1,furia-vs-gambit-m3-mirage,de_mirage
1,8150,34.742188,7.656120e+16,interz,Gambit,T,90.629951,-2367.156250,-39.968750,155.0,...,True,False,Ax1Le,Gambit,7.656120e+16,False,p250,1,furia-vs-gambit-m3-mirage,de_mirage
2,8889,40.515625,7.656120e+16,interz,Gambit,T,76.037605,-2083.767334,-35.968750,47.0,...,True,False,KSCERATO,FURIA,7.656120e+16,False,p250,1,furia-vs-gambit-m3-mirage,de_mirage
3,11856,63.695312,7.656120e+16,Hobbit,Gambit,T,278.709717,-1496.237915,-175.968750,60.0,...,True,False,arT,FURIA,7.656120e+16,False,USP-S,1,furia-vs-gambit-m3-mirage,de_mirage
4,12236,66.664062,7.656120e+16,yuurih,FURIA,CT,-504.352875,-400.609680,-168.368683,40.0,...,False,False,VINI,FURIA,7.656120e+16,False,USP-S,1,furia-vs-gambit-m3-mirage,de_mirage


In [142]:
len(killData.AttackerAreaName.value_counts())

100

In [143]:
damageData.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'Weapon', 'HpDamage', 'HpDamageTaken', 'ArmorDamage',
       'ArmorDamageTaken', 'HitGroup', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [144]:
roundData.columns

Index(['RoundNum', 'MatchId', 'MapName', 'StartTick', 'FreezeTimeEnd',
       'EndTick', 'EndOfficialTick', 'TScore', 'CTScore', 'WinningSide',
       'WinningTeam', 'LosingTeam', 'RoundEndReason', 'CTStartEqVal',
       'CTBuyType', 'TStartEqVal', 'TBuyType'],
      dtype='object')

In [145]:
roundData.WinningTeam.value_counts()

Gambit                  176
Gambit Esports           42
Heroic                   19
HAVU                     16
Virtus.pro Parimatch     16
forZe                    12
Virtus.pro               11
NAVI GG.BET              11
Nemiga Gaming            10
K23                      10
FURIA                     9
Spirit                    9
Winstrike                 9
FURIA eSports             6
Complexity                5
Team One                  5
Name: WinningTeam, dtype: int64

In [146]:
roundsPlayed = 0
for x in roundData.MatchId.unique():
    roundsPlayed += roundData[roundData['MatchId'] == x].RoundNum.iloc[-1]

In [147]:
roundsPlayed

340

In [148]:
killData.AttackerName.value_counts().index

Index(['nafany', 'Ax1Le', 'interz', 'Hobbit', 'sh1ro', 'sh1r0', 'SH1R0',
       'Ax1Le.VulkanBet', 'HObbit', 'HObbit.', 'HObbit.VulkanBet', 'stavn',
       'nafany.VulkanBet', 'KSCERATO', 'sh1ro.VulkanBet', 'interz.VulkanBet',
       'cadiaN', '-Ax1Le-', 'VINI', 'arT', 'yuurih', 'electronic',
       'xsepower x VulkanBET', 'Jame', 'sLowi', 'junior', 'SANJI',
       'FL1T x LUKOIL', 'xseveN', 'elian', 'Qikert', 'ZOREE', 'TeSeS',
       'Aerial', 'Boombl4', 'sdy.Parimatch', 'refrezh', 'Mir.Parimatch',
       'RUSH', 'kade0 1XBET', 'chopper.Parimatch', 'Jyo', 'Jame * Parimatch',
       'facecrack x LUKOIL', 'magixx.Parimatch', 's1mple', 'lollipop21k',
       'buster * Parimatch', 'buster', 'Maluk3', 'jemi',
       'YEKINDAR * Parimatch', 'mou 1XBET', 'almazer x LUKOIL', 'YEKINDAR',
       'SANJI * Parimatch', 'n0rb3r7', 'Lack1', 'Xo66uT', 'boX', 'sjuush',
       'k0nfig', 'pesadelo', 'malbsMd', 'niko ', 'nealan', 'mds',
       'Jerry x VulkanBET', 'b1t', 'Perfecto', 'n0rb3r7 1XBET', 'prt'

In [149]:
killData.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'AssisterSteamId', 'AssisterName', 'AssisterTeam',
       'AssisterSide', 'AssisterX', 'AssisterY', 'AssisterZ', 'AssisterAreaId',
       'AssisterAreaName', 'IsWallbang', 'IsFlashed', 'IsHeadshot', 'IsTrade',
       'PlayerTradedName', 'PlayerTradedTeam', 'PlayerTradedSteamId',
       'IsFirstKill', 'Weapon', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [150]:
shiro_names = ['sh1ro', 'sh1r0', 'SH1R0','sh1ro.VulkanBet']
hobbit_names = ['Hobbit','HObbit', 'HObbit.',  'HObbit.VulkanBet']
interz_names = ['interz','interz.VulkanBet']
nafany_names = ['nafany','nafany.VulkanBet']
ax1le_names = ['-Ax1Le-', 'Ax1Le', 'Ax1Le.VulkanBet']

# Kills + Damages

In [151]:
sh1roKills = killData[(killData['AttackerName'].isin(shiro_names)) |(killData['VictimName'].isin(shiro_names))]
hobbitKills = killData[(killData['AttackerName'].isin(hobbit_names)) |(killData['VictimName'].isin(hobbit_names))]
interzKills = killData[(killData['AttackerName'].isin(interz_names)) |(killData['VictimName'].isin(interz_names))]
nafanyKills = killData[(killData['AttackerName'].isin(nafany_names)) |(killData['VictimName'].isin(nafany_names))]
ax1leKills = killData[(killData['AttackerName'].isin(ax1le_names)) |(killData['VictimName'].isin(ax1le_names))]

In [152]:
damageData.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'Weapon', 'HpDamage', 'HpDamageTaken', 'ArmorDamage',
       'ArmorDamageTaken', 'HitGroup', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [153]:
sh1roDamages = damageData[(damageData['AttackerName'].isin(shiro_names)) |(damageData['VictimName'].isin(shiro_names))]
hobbitDamages = damageData[(damageData['AttackerName'].isin(hobbit_names)) |(damageData['VictimName'].isin(hobbit_names))]
interzDamages = damageData[(damageData['AttackerName'].isin(interz_names)) |(damageData['VictimName'].isin(interz_names))]
nafanyDamages = damageData[(damageData['AttackerName'].isin(nafany_names)) |(damageData['VictimName'].isin(nafany_names))]
ax1leDamages = damageData[(damageData['AttackerName'].isin(ax1le_names)) |(damageData['VictimName'].isin(ax1le_names))]

## Sh1ro

In [154]:
#Sh1ro's Weapon distirbution
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


AWP             54.9%
AK-47           10.9%
Desert Eagle     7.7%
USP-S            7.4%
p250             6.0%
SSG 08           3.2%
AUG              2.5%
M4A4             1.4%
SG 553           1.4%
Tec-9            1.1%
M4A1             0.7%
FAMAS            0.7%
Glock-18         0.7%
Galil AR         0.4%
Molotov          0.4%
Five-SeveN       0.4%
Knife            0.4%
Name: Weapon, dtype: object

In [155]:
#sh1r0's weapon distribution on ct
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'CT'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AWP             62.1%
USP-S           13.7%
Desert Eagle     8.1%
SSG 08           4.0%
AUG              3.2%
M4A4             3.2%
M4A1             1.6%
FAMAS            1.6%
p250             0.8%
Five-SeveN       0.8%
AK-47            0.8%
Name: Weapon, dtype: object

In [156]:
#sh1ro's weapon distribution on T
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'T'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AWP             49.4%
AK-47           18.8%
p250            10.0%
Desert Eagle     7.5%
SG 553           2.5%
SSG 08           2.5%
USP-S            2.5%
Tec-9            1.9%
AUG              1.9%
Glock-18         1.2%
Knife            0.6%
Galil AR         0.6%
Molotov          0.6%
Name: Weapon, dtype: object

In [157]:
#sh1ro's frequency distribution for location he was at when he got a kill
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


Ticketbooth                  9.5%
Top Balcony                  5.3%
T Palace Stairs Peek         4.9%
Default A                    4.2%
CT Side Apps                 4.2%
                             ... 
Van Barrel Peek              0.4%
Top Mid towards Apps         0.4%
Close Right Market Window    0.4%
Sandwhich                    0.4%
Top Connector                0.4%
Name: AttackerAreaName, Length: 67, dtype: object

In [158]:
#shr1ro's frequency distribution for location he was at when he got a kill as a CT
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'CT'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Ticketbooth                               18.5%
Bottom Stairs                              5.6%
Snipers Nest                               5.6%
Stairs Palace Peek                         5.6%
Default A                                  4.8%
Snipers Palace Peek                        4.0%
Close Market door                          4.0%
Right Side Snipers Nest                    3.2%
Triple Box                                 3.2%
Deep Jungle                                3.2%
Back Left B Site From T's' Perspective     3.2%
Right Side Connector                       3.2%
Deep CT                                    3.2%
Catwalk Plant B Site                       2.4%
Market Window Peek                         2.4%
Catwalk T Aparments Peek                   2.4%
CT Left Side                               1.6%
Elevated Snipers Awp Peek                  1.6%
Leftside Connector                         1.6%
Deep Palace                                1.6%
A Site Exposed to Stairs Connector Cat  

In [159]:
#sh1r0's frequency distribution of hislocation when he's creating a pick as a ct
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'CT'][sh1roKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Ticketbooth                               17.9%
Right Side Snipers Nest                   14.3%
Snipers Nest                              10.7%
Right Side Connector                       7.1%
Bottom Stairs                              7.1%
Elevated Snipers Awp Peek                  7.1%
Deep Palace                                7.1%
Back Right B Site from T's Perspective     7.1%
Default A                                  3.6%
A Site Exposed to Stairs Connector Cat     3.6%
Underpass                                  3.6%
Market Window Peek                         3.6%
Stairs Palace Peek                         3.6%
Bottom Mid Ladder Room Peek                3.6%
Name: AttackerAreaName, dtype: object

In [160]:
#sh1r0's frequency distribution of hislocation when he's creating a pick as a ct with the awp
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'CT'][sh1roKills['IsFirstKill'] == True][sh1roKills['Weapon'] == 'AWP'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Right Side Snipers Nest                   18.2%
Ticketbooth                               13.6%
Elevated Snipers Awp Peek                  9.1%
Deep Palace                                9.1%
Right Side Connector                       9.1%
Snipers Nest                               9.1%
Back Right B Site from T's Perspective     9.1%
Bottom Stairs                              4.5%
Underpass                                  4.5%
Market Window Peek                         4.5%
Stairs Palace Peek                         4.5%
Bottom Mid Ladder Room Peek                4.5%
Name: AttackerAreaName, dtype: object

In [ ]:
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'CT'][sh1roKills['IsFirstKill'] == True][sh1roKills['Weapon'] == 'AWP']

In [161]:
#shr1ro's frequency distribution for location he was at when he got a kill as a T
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'T'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Top Balcony                               9.4%
T Palace Stairs Peek                      8.8%
CT Side Apps                              7.5%
Left Side Platform                        6.2%
Close Left CT A Ramp                      5.0%
T Deep Apps Underpass/Apps Peek           3.8%
Default A                                 3.8%
Deep Palace                               3.8%
Behind Big Boxes                          3.1%
A Ramp                                    3.1%
Catwalk T Aparments Peek                  3.1%
Ticketbooth                               2.5%
B Site Default Plant                      2.5%
Top Mid Left Side                         2.5%
Top Mid Boxes Left Side Peek              1.9%
T Spawn A Ramp                            1.9%
Back Left B Site From T's' Perspective    1.2%
Firebox                                   1.2%
Back Right B Site from T's Perspective    1.2%
Top Mid Right Side                        1.2%
Tetris                                    1.2%
Mid Close Con

In [162]:
#shr1ro's frequency distribution for location he was at when he got an opening kill as a T
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'T'][sh1roKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Top Mid Left Side                  12.5%
Deep Palace                        12.5%
T Deep Apps Underpass/Apps Peek    12.5%
CT Side Apps                       12.5%
Left Side Platform                  6.2%
T Spawn A Ramp                      6.2%
Deep T Apps                         6.2%
Catwalk T Aparments Peek            6.2%
Top Mid Right Side                  6.2%
Mid Close Connector                 6.2%
Jungle Plant                        6.2%
Underpass                           6.2%
Name: AttackerAreaName, dtype: object

In [163]:
#shr1ro's frequency distribution for location he was at when he got a kill as a T with the awp
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'T'][sh1roKills['Weapon'] == 'AWP'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


T Palace Stairs Peek                      12.7%
Left Side Platform                         7.6%
CT Side Apps                               6.3%
Top Balcony                                6.3%
Top Mid Left Side                          5.1%
Default A                                  5.1%
Catwalk T Aparments Peek                   5.1%
Close Left CT A Ramp                       3.8%
T Deep Apps Underpass/Apps Peek            3.8%
Deep Palace                                3.8%
Top Mid Boxes Left Side Peek               2.5%
Mid Close Connector                        2.5%
Top Mid Right Side                         2.5%
Bottom Connector                           2.5%
T Spawn A Ramp                             2.5%
A Ramp                                     2.5%
Tetris                                     2.5%
Firebox                                    2.5%
Close A Ramp                               1.3%
Catwalk Connector Peek                     1.3%
T Spawn towards Apps                    

In [164]:
#shr1ro's frequency distribution for location he was at when he got an opening kill as a T with the awp
sh1roKills[sh1roKills['AttackerName'].isin(shiro_names)][sh1roKills['AttackerSide'] == 'T'][sh1roKills['IsFirstKill'] == True][sh1roKills['Weapon'] == 'AWP'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Top Mid Left Side                  16.7%
T Deep Apps Underpass/Apps Peek    16.7%
Deep Palace                        16.7%
Left Side Platform                  8.3%
Deep T Apps                         8.3%
Catwalk T Aparments Peek            8.3%
Top Mid Right Side                  8.3%
Mid Close Connector                 8.3%
T Spawn A Ramp                      8.3%
Name: AttackerAreaName, dtype: object

In [167]:
#kill death differenitals and damage differentiasl broken down by locatiuon  as a ct
print(prettyPrintPlayerDamageTable(sh1roDamages, sh1roKills, shiro_names, 'CT'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| Catwalk Connector Peek                 |       1 |        0 |         1 | 12             | 0              | 12                    |
| Deep Palace                            |       2 |        0 |         2 | 200            | 0              | 200                   |
| T Deep Apps Underpass/Apps Peek        |       0 |        0 |         0 | 0              | 0              | 0                     |
| Underneath B apps plat                 |       0 |        1 |        -1 | 0              | 100            | -100                  |
| T Palace Stairs Peek                   |       0 |        0 |         0 | 0              | 0              | 0                     |
| A Site Exposed to Stairs Connector Cat |       2 |        0 

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [168]:
#kill death differenitals and damage differentiasl broken down by locatiuon  as a T
print(prettyPrintPlayerDamageTable(sh1roDamages, sh1roKills, shiro_names, 'T'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| Catwalk Connector Peek                 |       1 |        1 |         0 | 100            | 100            | 0                     |
| Deep Palace                            |       6 |        2 |         4 | 578            | 545            | 33                    |
| T Deep Apps Underpass/Apps Peek        |       6 |        1 |         5 | 551            | 82             | 469                   |
| Underneath B apps plat                 |       2 |        0 |         2 | 98             | 0              | 98                    |
| T Palace Stairs Peek                   |      14 |        7 |         7 | 1578           | 924            | 654                   |
| A Site Exposed to Stairs Connector Cat |       1 |        2 

## Hobbit

In [169]:
#weapon usage distribution
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


AK-47           49.8%
M4A4            16.0%
MP9              7.6%
USP-S            7.2%
Galil AR         6.8%
Glock-18         3.4%
Desert Eagle     3.4%
Molotov          1.3%
Tec-9            0.8%
CZ75 Auto        0.8%
AWP              0.8%
HE Grenade       0.4%
UMP-45           0.4%
XM1014           0.4%
FAMAS            0.4%
MAC-10           0.4%
Name: Weapon, dtype: object

In [170]:
#weapon usage as a ct distribution
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)][hobbitKills['AttackerSide'] == 'CT'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


M4A4            33.6%
AK-47           22.4%
MP9             16.8%
USP-S           14.0%
Desert Eagle     4.7%
Galil AR         2.8%
CZ75 Auto        1.9%
UMP-45           0.9%
FAMAS            0.9%
HE Grenade       0.9%
XM1014           0.9%
Name: Weapon, dtype: object

In [171]:
#weapon usage as a t distribution
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)][hobbitKills['AttackerSide'] == 'T'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AK-47           72.3%
Galil AR        10.0%
Glock-18         6.2%
Molotov          2.3%
Desert Eagle     2.3%
Tec-9            1.5%
M4A4             1.5%
USP-S            1.5%
AWP              1.5%
MAC-10           0.8%
Name: Weapon, dtype: object

In [172]:
#distribution of location hobbit was at when he got a Kill 
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


Ticketbooth                  8.4%
Top Balcony                  5.9%
Sandwhich                    5.1%
Deep Palace                  4.6%
Tetris                       3.8%
                             ... 
Top STairs                   0.4%
Close Right Market Window    0.4%
Top Mid Right Side           0.4%
Catwalk Plant B Site         0.4%
T Spawn A Ramp               0.4%
Name: AttackerAreaName, Length: 63, dtype: object

In [173]:
#distribution of the locaiotn hobbit was at whne he got a kill as a CT
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)][hobbitKills['AttackerSide'] == 'CT'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Ticketbooth                               17.8%
Top Balcony                               11.2%
Sandwhich                                  6.5%
Under balcony                              5.6%
Default A                                  4.7%
A Ramp                                     4.7%
T Palace Stairs Peek                       3.7%
A Site                                     3.7%
Bottom Stairs                              3.7%
Firebox                                    2.8%
Close A Ramp                               2.8%
Tetris                                     2.8%
A Site Exposed to Stairs Connector Cat     2.8%
Triple Box                                 2.8%
CT Outside Market                          1.9%
B Bombsite Cross to Catwalk                1.9%
CT Left Side                               1.9%
CT Spawn                                   1.9%
Leftside Connector                         1.9%
Ninja                                      0.9%
Top Mid                                 

In [174]:
#distribution of the location hobbit was at when he got an opening kill as a CT
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)][hobbitKills['AttackerSide'] == 'CT'][hobbitKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Ticketbooth             41.2%
T Palace Stairs Peek    11.8%
Triple Box              11.8%
CT Left Side             5.9%
Close A Ramp             5.9%
Under balcony            5.9%
Underpass                5.9%
Top Balcony              5.9%
Stairs Palace Peek       5.9%
Name: AttackerAreaName, dtype: object

In [175]:
#distribution of the locaiotn hobbit was at whne he got a kill as a T
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)][hobbitKills['AttackerSide'] == 'T'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Deep Palace                               7.7%
Underpass                                 4.6%
Tetris                                    4.6%
Deep Catwalk                              4.6%
Bottom Stairs                             3.8%
Mid Close Connector                       3.8%
Sandwhich                                 3.8%
CT Side Apps                              3.8%
Bottom Connector                          3.1%
A Site Exposed to Stairs Connector Cat    3.1%
Chair                                     3.1%
Right Side Connector                      2.3%
Catwalk Connector Peek                    2.3%
A Site                                    2.3%
CT Catwalk                                2.3%
Back B Site Peeking Apps/Plat             2.3%
Deep Jungle                               2.3%
A Ramp                                    2.3%
CT Deep Catwalk Mid peek                  2.3%
market                                    1.5%
Top Mid Catwalk                           1.5%
Stairs Palace

In [176]:
#distribution of the location hobbit was at when he got an opening kill as a T
hobbitKills[hobbitKills['AttackerName'].isin(hobbit_names)][hobbitKills['AttackerSide'] == 'T'][hobbitKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Underpass                          18.2%
Deep T Apps Underpass STairs        9.1%
Mid Close Connector                 9.1%
Top Mid Left Side                   4.5%
Sandwhich                           4.5%
Top Mid Right Side                  4.5%
Top Mid Catwalk                     4.5%
Catwalk Connector Peek              4.5%
Tetris                              4.5%
Deep Palace                         4.5%
T Spawn A Ramp                      4.5%
Firebox                             4.5%
A Ramp                              4.5%
T Deep Apps Underpass/Apps Peek     4.5%
Chair                               4.5%
Bottom Stairs                       4.5%
Top Mid Boxes Left Side Peek        4.5%
Name: AttackerAreaName, dtype: object

In [177]:
#hobbit ct kd + damage differentials
print(prettyPrintPlayerDamageTable(hobbitDamages, hobbitKills, hobbit_names, 'CT'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| A Ramp                                 |       5 |        1 |         4 | 473            | 253            | 220                   |
| Sandwhich                              |       7 |        4 |         3 | 770            | 412            | 358                   |
| Market Window Peek                     |       1 |        0 |         1 | 79             | 26             | 53                    |
| Close A Ramp                           |       3 |        2 |         1 | 512            | 186            | 326                   |
| Tetris                                 |       3 |        2 |         1 | 300            | 221            | 79                    |
| A Site                                 |       4 |        2 

In [178]:
#hobbit t kd and damage differentials.
print(prettyPrintPlayerDamageTable(hobbitDamages, hobbitKills, hobbit_names, 'T'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| A Ramp                                 |       3 |       17 |       -14 | 772            | 1766           | -994                  |
| Sandwhich                              |       5 |        1 |         4 | 553            | 244            | 309                   |
| Market Window Peek                     |       0 |        0 |         0 | 0              | 0              | 0                     |
| Close A Ramp                           |       1 |        2 |        -1 | 214            | 159            | 55                    |
| Tetris                                 |       6 |        2 |         4 | 470            | 433            | 37                    |
| A Site                                 |       3 |        2 

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


## Interz

In [179]:
#weapon usage distribuition
interzKills[interzKills['AttackerName'].isin(interz_names)].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


AK-47           52.9%
M4A4            16.0%
USP-S            5.9%
FAMAS            4.8%
Galil AR         4.8%
Glock-18         3.2%
Desert Eagle     2.7%
AUG              2.1%
Tec-9            1.6%
MP9              1.6%
AWP              1.1%
p250             1.1%
HE Grenade       0.5%
UMP-45           0.5%
Five-SeveN       0.5%
MAC-10           0.5%
Name: Weapon, dtype: object

In [180]:
#weapon usage distribution as ct
interzKills[interzKills['AttackerName'].isin(interz_names)][interzKills['AttackerSide'] == 'CT'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


M4A4            39.1%
USP-S           15.9%
AK-47           14.5%
FAMAS           13.0%
AUG              5.8%
MP9              2.9%
Desert Eagle     2.9%
UMP-45           1.4%
Galil AR         1.4%
AWP              1.4%
Five-SeveN       1.4%
Name: Weapon, dtype: object

In [181]:
#weapon usage distribution as a T
interzKills[interzKills['AttackerName'].isin(interz_names)][interzKills['AttackerSide'] == 'T'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AK-47           75.4%
Galil AR         6.8%
Glock-18         5.1%
Desert Eagle     2.5%
Tec-9            2.5%
M4A4             2.5%
p250             1.7%
HE Grenade       0.8%
MP9              0.8%
MAC-10           0.8%
AWP              0.8%
Name: Weapon, dtype: object

In [182]:
#Distribution of the location he was at when he got a kill
interzKills[interzKills['AttackerName'].isin(interz_names)].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


Top Balcony                               7.5%
Underneath B apps plat                    7.0%
Left Side Platform                        5.9%
T Palace Stairs Peek                      5.9%
Back Left B Site From T's' Perspective    5.3%
Sandwhich                                 4.3%
Deep Palace                               3.7%
Bench                                     3.7%
Van Barrel Peek                           3.2%
Deep T Apps                               3.2%
A Site                                    2.7%
Deep Jungle                               2.7%
Ticketbooth                               2.1%
Catwalk Plant B Site                      2.1%
Deep CT                                   2.1%
CT Catwalk                                2.1%
Forest                                    1.6%
CT Side Apps                              1.6%
Back B Site Peeking Apps/Plat             1.6%
CT Spawn                                  1.6%
T Spawn A Ramp                            1.6%
Bottom Connec

In [183]:
#distribution of the location interz was at when he got a kill as a CT
interzKills[interzKills['AttackerName'].isin(interz_names)][interzKills['AttackerSide'] == 'CT'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Back Left B Site From T's' Perspective    14.5%
Left Side Platform                        10.1%
Bench                                      8.7%
Underneath B apps plat                     7.2%
CT Catwalk                                 5.8%
Deep CT                                    4.3%
Deep T Apps                                4.3%
CT Spawn                                   4.3%
Close Market door                          2.9%
Deep Jungle                                2.9%
Back B Site Peeking Apps/Plat              2.9%
Right Side Platform                        2.9%
B site Market Plant                        2.9%
Ticketbooth                                2.9%
Bottom Connector                           2.9%
Catwalk Plant B Site                       2.9%
Top STairs                                 1.4%
Ladder Room Catwalk Side                   1.4%
Mid Close Connector                        1.4%
market                                     1.4%
CT Outside Market                       

In [184]:
#distribution of the location interz was at when he got a=n opening kill as a CT
interzKills[interzKills['AttackerName'].isin(interz_names)][interzKills['AttackerSide'] == 'CT'][interzKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Back Left B Site From T's' Perspective    57.1%
Left Side Platform                        14.3%
Back B Site Peeking Apps/Plat             14.3%
CT Side Apps                              14.3%
Name: AttackerAreaName, dtype: object

In [186]:
#distribution of the location interz was at when he got a=n opening kill as a T
interzKills[interzKills['AttackerName'].isin(interz_names)][interzKills['AttackerSide'] == 'T'][interzKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Deep Palace                   37.5%
Apartments Right CT Corner    12.5%
Deep T Apps                   12.5%
T Palace Stairs Peek           6.2%
Top Balcony                    6.2%
Left Side Platform             6.2%
Deep Catwalk                   6.2%
T Spawn A Ramp                 6.2%
CT Side Apps                   6.2%
Name: AttackerAreaName, dtype: object

In [187]:
print(prettyPrintPlayerDamageTable(interzDamages, interzKills, interz_names, 'CT'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| T Palace Stairs Peek                   |       0 |        0 |         0 | 0              | 0              | 0                     |
| Top Balcony                            |       0 |        0 |         0 | 145            | 0              | 145                   |
| A Site Exposed to Stairs Connector Cat |       1 |        1 |         0 | 50             | 126            | -76                   |
| B site Market Plant                    |       2 |        1 |         1 | 155            | 88             | 67                    |
| Default A                              |       0 |        0 |         0 | 0              | 13             | -13                   |
| T Deep apps                            |       0 |        0 

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [188]:
print(prettyPrintPlayerDamageTable(interzDamages, interzKills, interz_names, 'T'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| T Palace Stairs Peek                   |      11 |       16 |        -5 | 1512           | 1771           | -259                  |
| Top Balcony                            |      14 |        9 |         5 | 1486           | 1142           | 344                   |
| A Site Exposed to Stairs Connector Cat |       1 |        4 |        -3 | 29             | 374            | -345                  |
| B site Market Plant                    |       1 |        0 |         1 | 66             | 0              | 66                    |
| Default A                              |       1 |        6 |        -5 | 420            | 627            | -207                  |
| T Deep apps                            |       1 |        0 

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


## Nafany

In [189]:
#weapon usage
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


AK-47                 37.8%
M4A4                  21.8%
FAMAS                  6.1%
AUG                    5.0%
Desert Eagle           5.0%
USP-S                  4.6%
UMP-45                 3.4%
MAC-10                 3.4%
Galil AR               2.3%
p250                   2.3%
SG 553                 2.3%
AWP                    1.5%
HE Grenade             1.1%
Tec-9                  1.1%
Molotov                0.8%
Glock-18               0.8%
Incendiary Grenade     0.4%
MP9                    0.4%
Name: Weapon, dtype: object

In [190]:
#nafany wepaon usage as a ct
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)][nafanyKills['AttackerSide'] == 'CT'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


M4A4                  40.1%
AK-47                 11.7%
FAMAS                 10.9%
AUG                    9.5%
USP-S                  8.8%
UMP-45                 6.6%
SG 553                 2.9%
Galil AR               2.9%
HE Grenade             2.2%
Desert Eagle           1.5%
Tec-9                  0.7%
Incendiary Grenade     0.7%
AWP                    0.7%
MP9                    0.7%
Name: Weapon, dtype: object

In [191]:
#weapon usage as a t
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)][nafanyKills['AttackerSide'] == 'T'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AK-47           66.4%
Desert Eagle     8.8%
MAC-10           7.2%
p250             4.8%
AWP              2.4%
Molotov          1.6%
M4A4             1.6%
Glock-18         1.6%
Tec-9            1.6%
SG 553           1.6%
Galil AR         1.6%
FAMAS            0.8%
Name: Weapon, dtype: object

In [192]:
#location he was at when he got a kill
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


Mid Close Connector                       7.6%
Deep Jungle                               6.5%
Snipers Palace Peek                       4.6%
A Ramp                                    4.6%
Top Mid                                   3.8%
                                          ... 
market                                    0.4%
Left Side corner deep jungle              0.4%
Deep CT                                   0.4%
Top Mid Catwalk                           0.4%
Back Right B Site from T's Perspective    0.4%
Name: AttackerAreaName, Length: 70, dtype: object

In [193]:
#location nafany was at when he got a kill as a CT
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)][nafanyKills['AttackerSide'] == 'CT'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Mid Close Connector                       9.5%
Deep Jungle                               8.0%
Snipers Nest                              7.3%
Snipers Palace Peek                       7.3%
Bottom Stairs                             5.1%
Top Mid                                   5.1%
Leftside Connector                        4.4%
A Site Exposed to Stairs Connector Cat    4.4%
Ticketbooth                               3.6%
Triple Box                                2.9%
Default A                                 2.2%
Top Mid Left Side                         2.2%
Connector behind box                      2.2%
Right Side Connector                      2.2%
Bottom Connector                          2.2%
Close Market door                         1.5%
Under balcony                             1.5%
Deep T Apps                               1.5%
Firebox                                   1.5%
Back B Site Peeking Apps/Plat             1.5%
A Ramp                                    1.5%
Top STairs   

In [194]:
#location nafany was at when he got a first kill as a ct
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)][nafanyKills['AttackerSide'] == 'CT'][nafanyKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Mid Close Connector           29.6%
Snipers Nest                  11.1%
Bottom Connector               7.4%
Snipers Palace Peek            7.4%
Top Mid                        7.4%
Deep Jungle                    7.4%
Default A                      3.7%
Right Side Connector           3.7%
Right Side Snipers Nest        3.7%
Bottom Stairs                  3.7%
Apartments Right CT Corner     3.7%
Deep T Spawn T Ramp            3.7%
Leftside Connector             3.7%
Ticketbooth                    3.7%
Name: AttackerAreaName, dtype: object

In [195]:
#location nafany was at when he got a kill as a T
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)][nafanyKills['AttackerSide'] == 'T'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


A Ramp                                    8.0%
A Site                                    6.4%
Mid Close Connector                       5.6%
Deep Jungle                               4.8%
CT Side Apps                              4.0%
Right Side Connector                      3.2%
Deep Catwalk                              3.2%
Default A                                 3.2%
CT Deep Catwalk Mid peek                  3.2%
Ticketbooth                               3.2%
Bottom Connector                          3.2%
Close A Ramp                              3.2%
Top Connector                             2.4%
Bottom Stairs                             2.4%
Ladder Room Catwalk Side                  2.4%
Top STairs                                2.4%
Under balcony                             2.4%
Top Mid                                   2.4%
A Site Exposed to Stairs Connector Cat    2.4%
Top Mid Boxes Left Side Peek              1.6%
Van Barrel Peek                           1.6%
T Spawn A Ram

In [196]:
#location nafany was at when he got a first kill as a t
nafanyKills[nafanyKills['AttackerName'].isin(nafany_names)][nafanyKills['AttackerSide'] == 'T'][nafanyKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Mid Close Connector             12.5%
Top Mid                         12.5%
Ladder Room Catwalk Side         8.3%
A Ramp                           8.3%
Catwalk Connector Peek           4.2%
Deep Jungle                      4.2%
A Site                           4.2%
Top Mid Catwalk                  4.2%
Under balcony                    4.2%
Close A Ramp                     4.2%
Tetris                           4.2%
Top Mid Left Side                4.2%
Bottom Connector                 4.2%
Top Mid Boxes Left Side Peek     4.2%
T Spawn A Ramp                   4.2%
Bottom Stairs                    4.2%
Right Side Connector             4.2%
Catwalk Plant B Site             4.2%
Name: AttackerAreaName, dtype: object

In [197]:
#damage + kd differentials as a CT
print(prettyPrintPlayerDamageTable(nafanyDamages, nafanyKills, nafany_names, 'CT'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| Right Side Snipers Nest                |       1 |        1 |         0 | 121            | 132            | -11                   |
| market                                 |       1 |        0 |         1 | 142            | 0              | 142                   |
| Right Side Connector                   |       3 |        1 |         2 | 217            | 178            | 39                    |
| A Site                                 |       1 |        0 |         1 | 66             | 0              | 66                    |
| A Ramp                                 |       2 |        2 |         0 | 167            | 200            | -33                   |
| T Deep apps                            |       1 |        0 

In [198]:
shiro_names

['sh1ro', 'sh1r0', 'SH1R0', 'sh1ro.VulkanBet']

In [199]:
#damage + kd differentials as a T
print(prettyPrintPlayerDamageTable(nafanyDamages, nafanyKills, nafany_names, 'T'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| Right Side Snipers Nest                |       2 |        1 |         1 | 148            | 17             | 131                   |
| market                                 |       0 |        0 |         0 | 0              | 0              | 0                     |
| Right Side Connector                   |       4 |        1 |         3 | 316            | 252            | 64                    |
| A Site                                 |       8 |        5 |         3 | 1031           | 662            | 369                   |
| A Ramp                                 |      10 |        7 |         3 | 964            | 904            | 60                    |
| T Deep apps                            |       1 |        0 

## Ax1le

In [200]:
#weapon usage
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


AK-47           60.3%
M4A4            11.3%
USP-S            5.3%
FAMAS            4.0%
Tec-9            2.7%
Galil AR         2.7%
AUG              2.3%
Desert Eagle     2.3%
p250             1.7%
HE Grenade       1.3%
Glock-18         1.3%
UMP-45           1.0%
AWP              1.0%
SSG 08           0.7%
M4A1             0.7%
MAC-10           0.7%
MP9              0.3%
Molotov          0.3%
Name: Weapon, dtype: object

In [201]:
#weapon usage as a ct
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'CT'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AK-47           42.3%
M4A4            24.1%
USP-S           10.9%
FAMAS            8.8%
AUG              5.1%
UMP-45           2.2%
HE Grenade       2.2%
Desert Eagle     1.5%
SSG 08           1.5%
M4A1             1.5%
Name: Weapon, dtype: object

In [202]:
#distribution of weapon usgae as a t
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'T'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AK-47           75.5%
Tec-9            4.9%
Galil AR         4.9%
p250             3.1%
Desert Eagle     3.1%
Glock-18         2.5%
AWP              1.8%
MAC-10           1.2%
MP9              0.6%
USP-S            0.6%
HE Grenade       0.6%
Molotov          0.6%
M4A4             0.6%
Name: Weapon, dtype: object

In [203]:
#distribution of locaiotn ax1le is at when he gets a kill
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


Deep Catwalk                    15.3%
CT Side Apps                     6.0%
CT Catwalk                       5.3%
Catwalk T Aparments Peek         4.0%
B Bombsite Cross to Catwalk      3.7%
                                ...  
CT Spawn                         0.3%
Top Mid Boxes Left Side Peek     0.3%
Top Mid towards Apps             0.3%
Register Market                  0.3%
Stairs Palace Peek               0.3%
Name: AttackerAreaName, Length: 69, dtype: object

In [204]:
#distribution of location ax1le was at when he got a kill as a ct
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'CT'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Deep Catwalk                              31.4%
CT Catwalk                                 9.5%
Catwalk T Aparments Peek                   8.8%
CT Deep Catwalk Mid peek                   4.4%
Ladder Room Catwalk Side                   2.9%
Right Side Snipers Nest                    2.9%
Deep Jungle                                2.9%
Mid Close Connector                        2.2%
Arch                                       2.2%
CT Side Apps                               2.2%
Underneath B apps plat                     2.2%
Snipers Palace Peek                        1.5%
Bench                                      1.5%
Deep CT                                    1.5%
Firebox                                    1.5%
Catwalk Connector Peek                     1.5%
Back Left B Site From T's' Perspective     1.5%
Top Connector                              1.5%
Bottom Stairs                              1.5%
Ticketbooth                                1.5%
Underpass                               

In [205]:
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'T']


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName
23,41529,37.093750,7.656120e+16,Ax1Le,Gambit,T,-701.981689,-778.488708,-261.968750,133.0,...,False,False,KSCERATO,FURIA,7.656120e+16,False,Galil AR,3,furia-vs-gambit-m3-mirage,de_mirage
26,49878,40.226562,7.656120e+16,Ax1Le,Gambit,T,109.615761,-1420.176636,-175.968750,15.0,...,True,False,Hobbit,Gambit,7.656120e+16,False,AK-47,4,furia-vs-gambit-m3-mirage,de_mirage
30,62987,47.593750,7.656120e+16,Ax1Le,Gambit,T,-2043.782227,497.751678,-166.405701,51.0,...,True,False,None,None,NaN,True,Tec-9,5,furia-vs-gambit-m3-mirage,de_mirage
31,63299,50.031250,7.656120e+16,Ax1Le,Gambit,T,-2026.625244,369.252319,-159.968750,50.0,...,True,False,KSCERATO,FURIA,7.656120e+16,False,Tec-9,5,furia-vs-gambit-m3-mirage,de_mirage
43,91651,29.664062,7.656120e+16,Ax1Le,Gambit,T,-1628.958984,811.719604,-47.968750,20.0,...,True,False,None,None,NaN,True,AK-47,7,furia-vs-gambit-m3-mirage,de_mirage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,77093,57.007874,7.656120e+16,-Ax1Le-,Gambit,T,370.008881,-785.899414,-162.788467,774.0,...,False,False,elian,Winstrike,7.656120e+16,False,AK-47,5,winstrike-vs-gambit-m2-mirage,de_mirage
2265,130840,38.440945,7.656120e+16,-Ax1Le-,Gambit,T,-751.584045,-1637.730835,-169.341675,397.0,...,False,False,nafany,Gambit,7.656120e+16,False,Galil AR,9,winstrike-vs-gambit-m2-mirage,de_mirage
2266,132042,47.905512,7.656120e+16,-Ax1Le-,Gambit,T,-958.325378,-1469.916992,-167.968750,38.0,...,True,False,n0rb3r7,Winstrike,7.656120e+16,False,Galil AR,9,winstrike-vs-gambit-m2-mirage,de_mirage
2280,172426,107.779528,7.656120e+16,-Ax1Le-,Gambit,T,-1057.785278,295.986816,-159.968750,228.0,...,False,False,nickelback,Winstrike,7.656120e+16,False,AK-47,11,winstrike-vs-gambit-m2-mirage,de_mirage


In [206]:
ax1leKills[ax1leKills['VictimName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'T']

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName
134,324453,38.703125,7.656120e+16,junior,FURIA,T,-1922.015137,758.596741,-47.968750,359.0,...,False,False,KSCERATO,FURIA,7.656120e+16,False,AWP,22,furia-vs-gambit-m3-mirage,de_mirage
146,347970,101.000000,7.656120e+16,yuurih,FURIA,T,-1234.552368,-1241.450195,-167.968750,1090.0,...,False,False,sh1r0,Gambit,7.656120e+16,False,AK-47,23,furia-vs-gambit-m3-mirage,de_mirage
151,357647,50.585938,7.656120e+16,KSCERATO,FURIA,T,-1851.837280,688.108093,-47.968750,160.0,...,False,False,VINI,FURIA,7.656120e+16,False,AK-47,24,furia-vs-gambit-m3-mirage,de_mirage
154,366175,30.828125,7.656120e+16,yuurih,FURIA,T,-1037.843872,-671.085754,-263.968750,1325.0,...,True,False,None,None,NaN,True,AK-47,25,furia-vs-gambit-m3-mirage,de_mirage
269,260160,56.779528,7.656120e+16,poizon,Complexity,T,357.678802,289.942780,-260.031647,82.0,...,True,False,RUSH,Complexity,7.656120e+16,False,Desert Eagle,17,gambit-vs-complexity-m2-mirage,de_mirage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,253138,73.251969,7.656120e+16,malbsMd,Team One,T,-107.439087,-1926.666016,-167.968750,460.0,...,False,False,sh1r0,Gambit,7.656120e+16,False,M4A1,17,teamone-vs-gambit-m2-mirage,de_mirage
2196,275488,90.228346,7.656120e+16,Maluk3,Team One,T,-642.452881,-898.969788,-234.480164,134.0,...,False,False,malbsMd,Team One,7.656120e+16,False,AWP,19,teamone-vs-gambit-m2-mirage,de_mirage
2309,249751,28.125984,7.656120e+16,nickelback,Winstrike,T,-1009.848511,-342.732971,-364.598541,2441.0,...,False,False,n0rb3r7,Winstrike,7.656120e+16,False,Glock-18,16,winstrike-vs-gambit-m2-mirage,de_mirage
2323,281300,71.763780,7.656120e+16,nealan,Winstrike,T,-1597.663086,769.796326,-47.968750,116.0,...,False,False,interz,Gambit,7.656120e+16,False,Galil AR,18,winstrike-vs-gambit-m2-mirage,de_mirage


In [207]:
len(ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'T'][ax1leKills['IsFirstKill'] == True])

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


18

In [208]:
len(ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'T'])

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


163

In [209]:
#Location ax1le was at when he got an opening kill as a CT
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'CT'][ax1leKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Deep Catwalk                65.0%
Catwalk T Aparments Peek    10.0%
Ladder Room Catwalk Side     5.0%
Mid Close Connector          5.0%
CT Catwalk                   5.0%
Arch                         5.0%
CT Deep Catwalk Mid peek     5.0%
Name: AttackerAreaName, dtype: object

In [210]:
#distribution of location ax1le was at when he got a kill as a t
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'T'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


CT Side Apps                              9.2%
B Bombsite Cross to Catwalk               6.7%
Left Side Platform                        4.9%
Deep T Apps Underpass STairs              4.3%
Deep Jungle                               4.3%
Deep T Apps                               3.7%
Ticketbooth                               3.7%
Back B Site Peeking Apps/Plat             3.1%
Underpass                                 2.5%
Default A                                 2.5%
A Site Exposed to Stairs Connector Cat    2.5%
A Site                                    2.5%
Tetris                                    2.5%
Top Mid Left Side                         2.5%
A Ramp                                    2.5%
T Deep Apps Underpass/Apps Peek           2.5%
Van Barrel Peek                           2.5%
B site Market Plant                       1.8%
Triple Box                                1.8%
Catwalk Plant B Site                      1.8%
Top Van                                   1.8%
T Palace Stai

In [211]:
#Location ax1le was at when he got an opening kill as a T
ax1leKills[ax1leKills['AttackerName'].isin(ax1le_names)][ax1leKills['AttackerSide'] == 'T'][ax1leKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Deep T Apps Underpass STairs       11.1%
B Bombsite Cross to Catwalk        11.1%
Underpass                          11.1%
Deep T Apps                        11.1%
CT Side Apps                       11.1%
Top Mid Left Side                  11.1%
Top Mid towards Apps                5.6%
Tetris                              5.6%
Bench Closer to Tribox              5.6%
A Site                              5.6%
Left Side Platform                  5.6%
T Deep Apps Underpass/Apps Peek     5.6%
Name: AttackerAreaName, dtype: object

In [212]:
#kd + damage differentials as ct
print(prettyPrintPlayerDamageTable(ax1leDamages, ax1leKills, ax1le_names, 'CT'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| Arch                                   |       3 |        2 |         1 | 469            | 200            | 269                   |
| Snipers Nest                           |       0 |        0 |         0 | 50             | 0              | 50                    |
| Mid Close Connector                    |       3 |        1 |         2 | 420            | 192            | 228                   |
| Close A Ramp                           |       0 |        0 |         0 | 17             | 13             | 4                     |
| Ticketbooth                            |       2 |        5 |        -3 | 272            | 463            | -191                  |
| Left Side Platform                     |       0 |        1 

In [213]:
#kd + damage differntials for ax1le as a t
print(prettyPrintPlayerDamageTable(ax1leDamages, ax1leKills, ax1le_names, 'T'))

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


| Places                                 |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   |
|----------------------------------------|---------|----------|-----------|----------------|----------------|-----------------------|
| Arch                                   |       0 |        2 |        -2 | 98             | 145            | -47                   |
| Snipers Nest                           |       0 |        0 |         0 | 12             | 0              | 12                    |
| Mid Close Connector                    |       3 |        6 |        -3 | 315            | 630            | -315                  |
| Close A Ramp                           |       2 |        3 |        -1 | 316            | 307            | 9                     |
| Ticketbooth                            |       6 |        3 |         3 | 475            | 389            | 86                    |
| Left Side Platform                     |       8 |        7 